In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
 
 
# 读取原始数据，并划分训练集和测试集
raw_data = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32)
X = raw_data[:, :-1]
y = raw_data[:, [-1]]
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.25)     #8:2比较好
Xtest = torch.from_numpy(Xtest)  # array转torch，才能送到pytorch里面
Ytest = torch.from_numpy(Ytest)

In [5]:
Xtest

tensor([[-0.8824,  0.0352,  0.3115,  ..., -0.4218, -0.6473, -0.9667],
        [-0.0588,  0.1055,  0.2459,  ..., -0.1714, -0.8642,  0.2333],
        [-0.2941,  0.0000,  0.1148,  ...,  0.1624, -0.4458, -0.3333],
        ...,
        [-0.5294,  0.2563,  0.3115,  ..., -0.0373, -0.6089, -0.8000],
        [-0.5294, -0.0452, -0.0164,  ...,  0.0551, -0.8241, -0.7667],
        [-0.5294,  0.2362,  0.3115,  ..., -0.0462, -0.6883, -0.5667]])